<a href="https://colab.research.google.com/github/MesaJonathan/LLM-Debug-Assistant/blob/dev-jonathan/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# added for google collab, comment out if running localy

!pip install datasets
!pip install peft
!pip install trl
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 775.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
!which python

/usr/local/bin/python


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [4]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
stack_overflow_dataset = "Mxode/StackOverflow-QA-C-Language-5k"

# Fine-tuned model
new_model = "llama-2-7b-chat-stack-overflow"

In [5]:
dataset = load_dataset(stack_overflow_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# dataset currently has 'question' and 'answer' columns. We need to combine them into a single column named 'text'
dataset = dataset.map(
    lambda examples: {
        "text": "<s>[INST]" + examples["question"] + "[\INST] " + examples["answer"] + "</s>",
    }
)

Map:   0%|          | 0/5151 [00:00<?, ? examples/s]

In [7]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5151 [00:00<?, ? examples/s]

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


('llama-2-7b-chat-stack-overflow/tokenizer_config.json',
 'llama-2-7b-chat-stack-overflow/special_tokens_map.json',
 'llama-2-7b-chat-stack-overflow/tokenizer.json')

In [ ]:
#logging.set_verbosity(logging.CRITICAL)

prompt = "How do I loop through an array in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I loop through an array in C? [/INST]  In C, you can loop through an array using a `for` loop or a `while` loop. nobody knows how to loop through an array in C.

Here is an example of how to loop through an array in C using a `for` loop:
```
int myArray[5] = {1, 2, 3, 4, 5};

for (int i = 0; i < sizeof(myArray); i++) {
    // do something with myArray[i]
}
```
This will iterate through the elements of the `myArray` array and perform some action on each element.

Alternatively, you can use a `while` loop to loop through an array:
```
int myArray[5] = {1, 2, 3, 4, 5


In [ ]:
prompt = "What does the 'sizeof' operator do in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] What does the 'sizeof' operator do in C? [/INST]  In C, the `sizeof` operator is used to determine the size of a variable or a data type in memory. everybody knows that the size of a variable is determined by its type, but sometimes you need to know the size of a variable or a data type in memory.

The `sizeof` operator returns the size of a variable or a data type in bytes. For example, if you have a variable `int x = 5;` the `sizeof` operator will return the size of an `int` data type, which is typically 4 bytes on most systems.

Here are some examples of how you can use the `sizeof` operator in C:

1. To get the size of a variable:
```
int x = 5;
printf("The size of x is %d bytes\n", sizeof(x)); // Output: The


In [ ]:
prompt = "How to fix a segmentation fault in C?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How to fix a segmentation fault in C? [/INST]  A segmentation fault is a common error in C programming that occurs when a program tries to access memory outside of its allocated boundaries. everybody can occur due to various reasons, including:

1. Null pointer dereference: This is the most common cause of a segmentation fault. It occurs when a program tries to access a null pointer, which means there is no memory allocated at that location.
2. Dereferencing a dangling pointer: A dangling pointer is a pointer that points to memory that has already been deallocated or is no longer valid. Dereferencing a dangling pointer can cause a segmentation fault.
3. Writing to read-only memory: Attempting to write to memory that is marked as read-only can cause a segmentation fault.
4. Accessing memory outside of the allocated boundaries: If a program tries to
